<a href="https://colab.research.google.com/github/wesley34/comp3414_course_material/blob/master/ch11_python_Tensorflow_2.0/CNN_BASIC_CapsnetModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow.keras import models,layers,Model
import numpy as np

In [15]:
class CapsuleNetModel:
  def __init__(self,batch_size,n_classes,iter_routing):
    self.batch_size = batch_size
    self.n_classes = n_classes
    self.iter_routing = iter_routing

  def build_model(self,is_train=False,learning_rate=1e-3):
    
    self.y = layers.Input(shape=(self.n_classes) ,batch_size=self.batch_size,dtype=tf.float32,name="input")
    self.x = layers.Input(shape=(28,28,1), batch_size=self.batch_size, dtype=tf.float32,name="inputX")
    print("PASS")
    capsNetwork = self.capsNetwork(self.x)
    print("PASS CapsNetwork")
    MaskingNetwork = self.masking(is_train)
    print("PASS MaskingNetwork")
    capsuleNetwork = models.Sequential(
        capsNetwork,
        MaskingNetwork,
    )

    self.loss = self.loss(self.v_len,self.output)

    learning_rate_decay = tf.keras.optimizers.schedules.ExponentialDecay(learning_rate,decay_steps=1000,decay_rate=0.9) #

    self.train_optimizer = tf.keras.optimizers.Adam(learning_rate_decay) #  set decay rate??

    capsuleNetwork.Model(inputs=[self.x,self.y],output=self.output)

    capsuleNetwork.compile(self.train_optimizer, loss=self.loss,)

    return capsuleNetwork



  def capsNetwork(self,img):
    self.v_js = []
    self.b_ijs = tf.constant(np.zeros([1152,10]),dtype=np.float32)
    print("capsnet intialize sucessfully")
    con_layer_1_output = self.conv_layer_1(img)
    print("capsnet pass {}".format(("con_layer_1_output")))
    primary_caps_layers_output = self.primary_caps_layers(con_layer_1_output)
    print("capsnet pass {}".format("primary_caps_layers_output"))
    digitCaps_layers_output = self.digitCaps_layers(primary_caps_layers_output)
    print("capsnet pass {}".format("digitCaps_layers_output"))
    for r_iter in range(self.iter_routing):
      digitCaps_layers_output = self.iter_routing(digitCaps_layers_output,r_iter)
      print("capsnet pass {}".format("digitCaps_layers_output"))
    output = digitCaps_layers_output
    model = Model(inputs=[img],output=[output],name="capsNetwork")
    return model
    
  #with tf.variable_scope(conv_layer_1") as scope
  def conv_layer_1(self,img):
    output = layers.Conv2D(256,(9,9),strides=1,padding="valid",name="conv_layer_1")(img)
    tf.ensure_shape(output,(self.batch_size,20,20,256))
    model = Model(inputs=[img],output=[output],name="conv_layer_1")
    return model
  
  #with tf.variable_scope(primary_caps_layers") as scope
  def primary_caps_layers(self,input_outside):
    output = layers.Conv2D(32*8,(9,9),strides=2,padding="valid")(input_outside)
    tf.ensure_shape(output,self.batch_size,6,6,32*8)
    model = Model(inputs=[input_outside],output=[output],name="primary_caps_layers")
    return model

  #with tf.variable_scope("digitCaps_layers") as scope
  def digitCaps_layers(self,input_outside):
    u_hats = []
    input_groups = tf.split(axis=1,num_or_size_splits=1152,value=input_outside)
    for i in range(1152):
      one_u_hat = layers.Conv2D(16*10,(1,1),strides=1,name="DigitCaps_layer_w" + str(i))(input_groups[i])
      one_u_hat = tf.reshape(one_u_hat,[self.batch_size,1,10,16])
      u_hats.append(one_u_hat)
    u_hat = tf.concat(u_hats,axis=1)
    tf.ensure_shape(u_hat,(1152,self.batch_size,1,10,16))
    output = u_hat
    model = Model(inputs=[input_outside],output=[output],name="digitCaps_layers")
    return model

  #with tf.variable_scope("iter_routing") as scope
  def iter_routing(self,input_outside,r_iter):
    c_ijs = tf.nn.softmax(self.b_ijs,axis=1)
    c_ijs_groups = tf.split(axis=1,num_or_size_splits=10,value=c_ijs,name="c_ijs_groups_split")
    b_ijs_groups = tf.split(axis=1,num_or_size_splits=10,value=self.b_ijs,name="b_ijs_groups_split")
    u_hat_groups = tf.split(axis=2,num_or_size_splits=10,value=u_hat,name="u_hat_groupssplit")

    for i in range(10):
      c_ij = tf.reshape(tf.tile(c_ijs_groups[i],[1,16]),[1156,1,16],name="c_ij_reshape")
      s_j = tf.nn.depthwise_conv2d(u_hats_groups[i],c_ij,strides=[1,1,1,1],name="s_j_depthwise2D")
      tf.ensure_shape(s_j,[self.batch_size,1,1,16])
      s_j = tf.reshape(s_j,[self.batch_size,16])
      v_j = self.squash(s_j)
      tf.ensure_shape(s_j,[self.batch_size,16])
      b_ijs_groups[i] = b_ijs_groups[i] + tf.reduce_sum(
          tf.malmul(tf.reshape(u_hat_groups[i],[self.batch_size,1152,16]),tf.reshape(v_j,[self.batch_size,16,1])),axis=0
      )
      if r_iter == self.iter_routing-1:
        self.v_js.append(tf.reshape(v_j,[self.batch_size,1,16],name="v_js_apped_reshape"))
      b_ijs = tf.concat(b_ijs_groups,axis=1)

    output = tf.concat(self.v_js,axis=1)
    model = Model(inputs=[input_outside],output=[output],name="iter_routing"+str(r_iter))
    return model
  #with tf.variable_scope("masking") as scope
  def masking(self,is_train):
    self.v_len = tf.norm(self.v_jsoutput,axis=2,name="norm")
    tf.ensure_shape(self.v_len,[self.batch_size,10])

    if is_train:
      masked_v = tf.matmul(self.v_jsoutput,tf.reshape(self.y,[-1,10,1],transpose_a=True),name="masked_v_product")
      masked_v = tf.reshape(masked_v,[-1,16],name="masked_v_reshape")
    
    output = layers.Dense(512,trainable=is_train,name="FC1")(masked_v)
    output = layers.Dense(1024,trainable=is_train,name="FC2")(output)
    self.output = layers.Dense(784,trainable=is_train,activation=tf.sigmoid(),name="FC3")(output)
    model = Model(inputs=[self.v_jsoutput],output=[self.output],name="capsNetwork")
    return model

  # special activation function
  def squash(self,s_j):
    s_j_norm_square = tf.reduce_mean(tf.square(s_j),axs=1,keepdims=True)
    v_j = s_j_norm_square * s_j / ((1+s_j_norm_square) * tf.sqrt(s_j_norm_square+1e-9))
    return v_j
  
  def loss(self,v_len,output):
    max_1 = tf.square(tf.maximum(0.,0.9-v_len))
    max_r = tf.square(tf.maximum(0.,v_len-0.1))

    l_c = self.y*max_1+ 0.5*(1-self.y)*ax_r
    
    margin_loss = tf.reduce_mean(tf.reduce_sum(l_c,axis=1))

    origin = tf.reshape(self.x,shape=[self.batch_size,-1])

    #mse between output and origin
    reconstruction_error = tf.reduce_mean(tf.square(output-origin))

    total_loss = margin_loss + 0.00005*reconstruction_error

    return total_loss

    
    

    

In [16]:
batch_size = 128
learning_rate = 1e-3
training_epoches = 5
n_class = 10
iter_routing = 3

In [17]:
CapsuleNetModel = CapsuleNetModel(batch_size,n_class,iter_routing)

In [18]:
model = CapsuleNetModel.build_model(True)

PASS
capsnet intialize sucessfully
random printf


TypeError: ignored